In [1]:
import json
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import joblib
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load intents data
with open('dataset.json', 'r') as file:
    intents_data = json.load(file)

# Convert the JSON data into a DataFrame
data = pd.DataFrame(columns=['text', 'label', 'responses'])
for intent in intents_data['intents']:
    for pattern in intent.get('patterns', []):
        response = intent.get('responses', [''])[0]
        data = data.append({'text': pattern, 'label': intent['tag'], 'responses': response}, ignore_index=True)

# Preprocessing dataset
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    return ' '.join(words)

data['processed_text'] = data['text'].apply(preprocess_text)

<ipython-input-2-000077b668e8>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'text': pattern, 'label': intent['tag'], 'responses': response}, ignore_index=True)
<ipython-input-2-000077b668e8>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'text': pattern, 'label': intent['tag'], 'responses': response}, ignore_index=True)
<ipython-input-2-000077b668e8>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'text': pattern, 'label': intent['tag'], 'responses': response}, ignore_index=True)
<ipython-input-2-000077b668e8>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = d

In [3]:
# Grid Search parameter to find optimal variable of C
param_grid = {
    'svm__C': [0.1, 1, 10, 100]
}

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(kernel='linear'))
])

# Use KFold with 5 splits
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=KFold(n_splits=5),error_score='raise')
grid_search.fit(data['processed_text'], data['label'])

# Save the trained model
joblib.dump(grid_search.best_estimator_, 'svm_model.pkl')


['svm_model.pkl']

In [ ]:
# Continuously take user input and provide responses
while True:
    # Get user input
    user_message = input("You: ")

    # Check if the user wants to exit the loop
    if user_message.lower() == 'exit' or 'hasthank' in user_message.lower() or 'goodbye' in user_message.lower():
        print("Goodbye!")
        break

    # Preprocess user input
    user_processed_text = preprocess_text(user_message)

    # Predict user message's label
    user_label = grid_search.best_estimator_.predict([user_processed_text])[0]

    # Calculate the Cosine Similarity score for each dataset with the same label
    similarities = cosine_similarity(grid_search.best_estimator_.named_steps['tfidf'].transform(data[data['label'] == user_label]['processed_text']),
                                     grid_search.best_estimator_.named_steps['tfidf'].transform([user_processed_text]))

    # Get data that has the highest score
    reply_data = data.loc[data['label'] == user_label].iloc[0]

    # Reply with data that has been obtained from Cosine Similarity process
    if similarities.max() >= 0.1:
        print("Bot:", reply_data['responses'])
    else:
        print("Bot: There are no information about the question. Please reiterate the question, thank you very much!")

    # Print similarity scores
    print("Similarity scores:", similarities)


You: What is cscs?
Bot: CSCS stands for College of Science and Computer Studies.
Similarity scores: [[1.        ]
 [0.86726684]]
You: who is the patron saint of cscs?
Bot: Blessed Arnold Jules-Nicolas Rèche is the patron of CSCS (College of Science and Computer Studies) at DLSU-D. His feast day is celebrated on October 23. He is known for his piety and discipline. Blessed Arnold Jules-Nicolas Rèche worked with other Brothers for the medical and spiritual care of wounded soldiers during the Franco-Prussian war. He is attributed with some cures and miracles.
Similarity scores: [[0.        ]
 [0.70985229]
 [0.        ]]
You: Can I punch someone in the face?
Bot: Physically attacking or assaulting any person inside or outside the campus, within the immediate vicinity of the University, or during official off-campus activities is considered a major offense (violent acts). The corrective actions for this offense are Corrective Action 3, 4, and 5.
Similarity scores: [[0.]
 [0.]
 [1.]]
